# Módulo 2: Lectura y Escritura de Archivos Parquet con Polars

Apache Parquet es un formato de archivo columnar de código abierto diseñado para el almacenamiento eficiente de datos y el análisis de grandes volúmenes. Sus principales ventajas sobre formatos basados en filas como CSV son:
- **Eficiencia de Almacenamiento:** Gracias a la compresión por columna y a los esquemas de codificación, los archivos Parquet suelen ser significativamente más pequeños que los CSV.
- **Rendimiento de Consulta:** Las consultas que solo necesitan un subconjunto de columnas pueden leer solo esas columnas, evitando la I/O innecesaria de datos no requeridos.
- **Preservación de Esquemas:** Parquet almacena el esquema de datos (tipos de datos de cada columna) dentro del propio archivo, lo que evita errores de inferencia de tipos al leerlos.

Polars tiene un soporte de primera clase para Parquet, utilizando el motor Apache Arrow por debajo. Las funciones principales son `pl.read_parquet()` para leer y `DataFrame.write_parquet()` para escribir.

In [ ]:
import polars as pl
import os

# Crear un DataFrame de ejemplo (similar al usado en ejemplos de CSV)
data = {
    "ID_Usuario": [1, 2, 3, 4, 5],
    "Nombre": ["Alicia", "Roberto", "Carlos", "Diana", "Eduardo"],
    "Email": ["a@example.com", "b@example.com", "c@example.com", "d@example.com", "e@example.com"],
    "Edad": [28, 34, 22, 45, None], # Valor nulo en Edad
    "Puntuacion": [8.5, 9.1, 7.7, None, 6.9], # Valor nulo en Puntuacion
    "Fecha_Registro": pl.date_range(pl.Date(2023, 1, 10), pl.Date(2023, 1, 14), "1d", eager=True),
    "Activo": [True, False, True, False, True]
}
df_ejemplo = pl.DataFrame(data)

print("DataFrame de Ejemplo:")
print(df_ejemplo)
print("\nGlimpse del DataFrame de Ejemplo:")
df_ejemplo.glimpse()

## Escribir a un Archivo Parquet (`DataFrame.write_parquet()`)

Para guardar un DataFrame en formato Parquet, se utiliza el método `write_parquet()`. Simplemente se le proporciona la ruta del archivo.
Polars (a través de Arrow) aplicará una compresión por defecto si está disponible (comúnmente `snappy`). Se puede especificar el algoritmo de compresión deseado mediante el parámetro `compression`.

In [ ]:
file_path_parquet_base = "df_ejemplo.parquet"
file_path_parquet_gzip = "df_ejemplo_gzip.parquet"
file_path_parquet_uncompressed = "df_ejemplo_uncompressed.parquet"

# 1. Escritura básica
# Polars usa 'zstd' por defecto si no se especifica y la feature está compilada, sino 'lz4', o 'uncompressed'.
# Para consistencia, vamos a especificar 'uncompressed' para el base y luego comparar.
df_ejemplo.write_parquet(file_path_parquet_uncompressed, compression='uncompressed')
print(f"DataFrame guardado en: {file_path_parquet_uncompressed} (sin compresión)")

# 2. Escritura especificando compresión 'snappy' (común y rápida)
# Nota: La disponibilidad de algoritmos de compresión puede depender de cómo se compiló Polars/PyArrow.
# 'snappy' es generalmente soportado.
try:
    df_ejemplo.write_parquet(file_path_parquet_base, compression='snappy')
    print(f"DataFrame guardado en: {file_path_parquet_base} (compresión snappy)")
except Exception as e:
    print(f"No se pudo guardar con snappy: {e}. Guardando sin compresión en su lugar.")
    df_ejemplo.write_parquet(file_path_parquet_base, compression='uncompressed') # Fallback

# 3. Escritura especificando compresión 'gzip' (buen ratio, más lento)
df_ejemplo.write_parquet(file_path_parquet_gzip, compression='gzip')
print(f"DataFrame guardado en: {file_path_parquet_gzip} (compresión gzip)")

# 4. (Opcional) Comparar tamaños de archivo
if os.path.exists(file_path_parquet_uncompressed):
    size_uncompressed = os.path.getsize(file_path_parquet_uncompressed)
    print(f"\nTamaño sin compresión: {size_uncompressed} bytes")

if os.path.exists(file_path_parquet_base):
    size_base = os.path.getsize(file_path_parquet_base)
    print(f"Tamaño con compresión por defecto/snappy: {size_base} bytes")

if os.path.exists(file_path_parquet_gzip):
    size_gzip = os.path.getsize(file_path_parquet_gzip)
    print(f"Tamaño con compresión gzip: {size_gzip} bytes")

## Leer desde un Archivo Parquet (`pl.read_parquet()`)

Para leer un archivo Parquet, se utiliza la función `pl.read_parquet()`. Una de las grandes ventajas es que Parquet almacena el esquema, por lo que Polars leerá los tipos de datos correctamente sin necesidad de inferencia compleja o especificación manual en la mayoría de los casos.

In [ ]:
# Usamos el archivo base (potencialmente con compresión snappy o sin compresión si snappy falló)
df_leido_parquet = pl.read_parquet(file_path_parquet_base)

print("\nDataFrame leído desde Parquet (base):")
print(df_leido_parquet)

print("\nGlimpse del DataFrame leído:")
df_leido_parquet.glimpse()

# Verificar igualdad
# El método `equals()` es estricto. Compara tipos de datos y valores, incluyendo cómo se manejan los NaN.
# Para floats, pequeñas diferencias de precisión podrían causar que `equals()` devuelva False.
# Para nulos, `equals` los trata como iguales si ambos son nulos.
print(f"\n¿Los DataFrames (original y leído) son iguales? {df_ejemplo.equals(df_leido_parquet)}")

## Selección de Columnas al Leer (`columns`)

Una ventaja significativa de los formatos columnares como Parquet es la capacidad de leer solo las columnas necesarias. Esto puede ahorrar una cantidad considerable de tiempo y memoria, especialmente con DataFrames anchos (muchas columnas).
El parámetro `columns` de `pl.read_parquet()` toma una lista de nombres de columna a leer.

In [ ]:
# Seleccionar la primera ('ID_Usuario') y tercera ('Email') columna por nombre
columnas_a_leer = [df_ejemplo.columns[0], df_ejemplo.columns[2]] 

df_leido_columnas = pl.read_parquet(file_path_parquet_base, columns=columnas_a_leer)

print(f"\nDataFrame leído con columnas seleccionadas ({columnas_a_leer}):")
print(df_leido_columnas)
df_leido_columnas.glimpse()

## Otras Opciones (Breve Mención)

Tanto `read_parquet` como `write_parquet` ofrecen opciones adicionales para escenarios más avanzados:

- **`n_rows` (para `read_parquet`):** Similar a `read_csv`, permite leer solo las primeras `N` filas del archivo Parquet. Útil para inspeccionar archivos grandes.
- **`use_pyarrow` (para ambos):** Por defecto, Polars utiliza su propio lector/escritor de Parquet implementado en Rust, que es generalmente muy rápido. Si se establece en `True`, se utilizará la implementación de PyArrow. Esto podría ser útil para compatibilidad o si se necesitan características específicas de PyArrow.
- **`pyarrow_options` (para ambos, si `use_pyarrow=True`):** Permite pasar un diccionario de opciones específicas para el motor PyArrow, como filtros de predicados (`filters`) durante la lectura para filtrar filas a nivel de I/O, o opciones de versión de Parquet al escribir.
- **`row_group_index` / `row_group_predicate` (para `read_parquet`):** Parquet organiza los datos en "row groups". Estas opciones permiten leer selectivamente solo ciertos grupos de filas, lo que puede ser muy eficiente.
- **`statistics=True` (para `write_parquet`):** Escribe metadatos estadísticos (min/max, conteo de nulos) para las columnas en el archivo Parquet. Esto puede ser usado por motores de consulta para optimizar aún más las lecturas (e.g., saltarse row groups que no cumplen un predicado).

## Limpieza de Archivos Creados

In [ ]:
archivos_a_eliminar = [
    file_path_parquet_base,
    file_path_parquet_gzip,
    file_path_parquet_uncompressed
]

print("\nLimpiando archivos generados...")
for f_path in archivos_a_eliminar:
    if os.path.exists(f_path):
        try:
            os.remove(f_path)
            print(f"Archivo '{f_path}' eliminado.")
        except OSError as e:
            print(f"Error al eliminar '{f_path}': {e.strerror}")
    else:
        print(f"Archivo '{f_path}' no encontrado (posiblemente ya eliminado o no creado).")